# Week 6: CNN Concepts & Trust Region Optimization

## Strategy: "Receptive Field" Optimization
Inspired by CNNs (Module 17), which build complex features from local receptive fields, we refine our strategy:
1. **Local Receptive Fields (Trust Regions):** Instead of optimizing over the whole [0, 1] space, we constrain the search to a small box around the current best point (e.g., $\pm 20\%$). This mimics how CNNs focus on local features.
2. **Pooling (Ensemble Averaging):** We continue using an ensemble of Neural Networks to "pool" predictions and reduce noise.
3. **Progressive Refinement:** We move from global exploration (early weeks) to fine-grained exploitation (Week 6).

In [1]:
import numpy as np
import warnings
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from scipy.optimize import minimize
import sys
import os

# Ensure we can import from src
sys.path.append(os.path.abspath('..'))
from src.utils import load_data

warnings.filterwarnings("ignore")
np.random.seed(46) # Week 6 Seed

print("Ready for Trust Region Optimization")

Ready for Trust Region Optimization


In [2]:
def suggest_next_point_trust_region(func_id, X_train, y_train):
    print(f"--- Optimizing Function {func_id} (Trust Region) ---")
    
    # 1. Preprocessing
    scaler_x = StandardScaler()
    X_scaled = scaler_x.fit_transform(X_train)
    
    scaler_y = StandardScaler()
    y_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()
    
    # 2. Train Ensemble (The "Filters")
    nn_ensemble = []
    seeds = [42, 101, 999]
    for seed in seeds:
        model = MLPRegressor(hidden_layer_sizes=(64, 32), activation='tanh', 
                             solver='lbfgs', max_iter=2000, random_state=seed)
        model.fit(X_scaled, y_scaled)
        nn_ensemble.append(model)
    
    # GP for Uncertainty
    kernel = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5) + WhiteKernel(noise_level=0.1)
    gp_model = GaussianProcessRegressor(kernel=kernel, normalize_y=False)
    gp_model.fit(X_scaled, y_scaled)
    
    # 3. Objective Function (Pooling)
    def objective_function(x):
        x_reshaped = x.reshape(1, -1)
        nn_preds = [m.predict(x_reshaped)[0] for m in nn_ensemble]
        avg_nn_pred = np.mean(nn_preds)
        gp_pred = gp_model.predict(x_reshaped)[0]
        
        # High Dim (Func 8) -> Trust GP more. Low Dim -> Trust NN.
        if func_id == 8:
            combined = 0.4 * avg_nn_pred + 0.6 * gp_pred
        else:
            combined = 0.7 * avg_nn_pred + 0.3 * gp_pred
        return -combined

    # 4. Define Trust Region (The "Receptive Field")
    # Instead of searching [0, 1], we search [best_x - radius, best_x + radius]
    best_idx = np.argmax(y_train)
    x_start_original = X_train[best_idx]
    x_start_scaled = scaler_x.transform(x_start_original.reshape(1, -1)).flatten()
    
    # Radius of 20% of the domain
    radius = 0.2 
    
    # Calculate bounds in SCALED space
    bounds_scaled = []
    for i in range(X_train.shape[1]):
        # Get scaler parameters for this dimension
        mean = scaler_x.mean_[i]
        scale = scaler_x.scale_[i]
        
        # Current best value in real space
        curr_val = x_start_original[i]
        
        # Trust Region: [curr - 0.2, curr + 0.2], clipped to [0, 1]
        lower_real = max(0.0, curr_val - radius)
        upper_real = min(1.0, curr_val + radius)
        
        # Convert these real trust bounds to scaled bounds
        lower_scaled = (lower_real - mean) / scale
        upper_scaled = (upper_real - mean) / scale
        
        bounds_scaled.append((lower_scaled, upper_scaled))
    
    # 5. Optimize within Trust Region
    res = minimize(fun=objective_function, 
                   x0=x_start_scaled, 
                   method='L-BFGS-B', 
                   bounds=bounds_scaled,
                   options={'maxiter': 100, 'eps': 1e-5})
    
    # 6. Inverse Transform
    x_optimized_scaled = res.x.reshape(1, -1)
    next_point = scaler_x.inverse_transform(x_optimized_scaled).flatten()
    next_point = np.clip(next_point, 0.0, 1.0)
    
    # Exploration Check: If we are stuck, force a small jump
    if np.linalg.norm(next_point - x_start_original) < 1e-4:
        print("   Local max reached. Applying 'Stride' exploration.")
        # Add random noise (like a stride in CNN)
        next_point += np.random.normal(0, 0.05, size=next_point.shape)
        next_point = np.clip(next_point, 0.0, 1.0)

    return next_point

In [4]:
submission_queries = {}
print(f"{'Func':<5} | {'Optimizing...'}")
print("-" * 30)

for func_id in range(1, 9):
    X_known, y_known = load_data(func_id)
    next_x = suggest_next_point_trust_region(func_id, X_known, y_known)
    submission_queries[func_id] = next_x

print("\n" + "="*30)
print("FORMATTED SUBMISSION OUTPUT")
print("="*30)

for func_id, x_val in submission_queries.items():
    formatted_str = "-".join([f"{val:.6f}" for val in x_val])
    print(f"function_number: {func_id}: {formatted_str}")

Func  | Optimizing...
------------------------------
--- Optimizing Function 1 (Trust Region) ---
--- Optimizing Function 2 (Trust Region) ---
--- Optimizing Function 3 (Trust Region) ---
--- Optimizing Function 4 (Trust Region) ---
--- Optimizing Function 5 (Trust Region) ---
   Local max reached. Applying 'Stride' exploration.
--- Optimizing Function 6 (Trust Region) ---
--- Optimizing Function 7 (Trust Region) ---
--- Optimizing Function 8 (Trust Region) ---

FORMATTED SUBMISSION OUTPUT
function_number: 1: 0.736372-0.761579
function_number: 2: 0.711417-0.927013
function_number: 3: 0.431836-0.574352-0.433932
function_number: 4: 0.454644-0.389414-0.365230-0.370203
function_number: 5: 1.000000-0.991192-0.996341-0.971717
function_number: 6: 0.350303-0.310764-0.408719-0.769796-0.088833
function_number: 7: 0.000000-0.330063-0.302456-0.234809-0.297204-0.698713
function_number: 8: 0.019886-0.184962-0.031335-0.213207-0.474043-0.402089-0.184597-0.546610
